# 모델 학습시키기

In [ ]:
!pip install transformers==4.50.0 datasets==3.5.0 huggingface_hub==0.29.0 -qqq

## 모델 학습에 사용할 연합뉴스 데이터셋 다운로드

In [ ]:
from datasets import load_dataset
klue_tc_train = load_dataset('klue', 'ynat', split='train')
klue_tc_eval = load_dataset('klue', 'ynat', split='validation')
klue_tc_train

In [ ]:
klue_tc_train[0]

In [ ]:
klue_tc_train.features['label'].names
# ['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']

## 실습에 사용하지 않는 불필요한 컬럼 제거

In [ ]:
klue_tc_train = klue_tc_train.remove_columns(['guid', 'url', 'date'])
klue_tc_eval = klue_tc_eval.remove_columns(['guid', 'url', 'date'])
klue_tc_train

## 카테고리를 문자로 표기한 label_str 컬럼 추가

In [ ]:
klue_tc_train.features['label']
# ClassLabel(names=['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치'], id=None)

klue_tc_train.features['label'].int2str(1)
# '경제'

klue_tc_label = klue_tc_train.features['label']

def make_str_label(batch):
  batch['label_str'] = klue_tc_label.int2str(batch['label'])
  return batch

klue_tc_train = klue_tc_train.map(make_str_label, batched=True, batch_size=1000)

klue_tc_train[0]
# {'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영', 'label': 3, 'label_str': '생활문화'}

## 학습/검증/테스트 데이터셋 분할

In [ ]:
train_dataset = klue_tc_train.train_test_split(test_size=10000, shuffle=True, seed=42)['test']
dataset = klue_tc_eval.train_test_split(test_size=1000, shuffle=True, seed=42)
test_dataset = dataset['test']
valid_dataset = dataset['train'].train_test_split(test_size=1000, shuffle=True, seed=42)['test']

## Trainer를 사용한 학습: (1) 준비

In [ ]:
import torch
import numpy as np
from transformers import (
    Trainer,
    TrainingArguments,
    AutoModelForSequenceClassification,
    AutoTokenizer
)

def tokenize_function(examples):
    return tokenizer(examples["title"], padding="max_length", truncation=True)

model_id = "klue/roberta-base"
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=len(train_dataset.features['label'].names))
tokenizer = AutoTokenizer.from_pretrained(model_id)

train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

## Trainer를 사용한 학습: (2) 학습 인자와 평가 함수 정의

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    push_to_hub=False,
    report_to="none"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": (predictions == labels).mean()}

## Trainer를 사용한 학습 - (3) 학습 진행

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.evaluate(test_dataset) # 정확도 0.84

## Trainer를 사용한 학습 - (4) 예측

In [ ]:
from transformers import pipeline

# 모델 저장
model.save_pretrained("./results")
tokenizer.save_pretrained("./results")

# 이미 파인튜닝된 모델 로드
model_path = "./results"
classifier = pipeline("text-classification", model=model_path, tokenizer="klue/roberta-base")

# 실제 라벨 이름 매핑
label_map = {
    "LABEL_0": "IT과학",
    "LABEL_1": "경제",
    "LABEL_2": "사회",
    "LABEL_3": "생활문화",
    "LABEL_4": "세계",
    "LABEL_5": "스포츠",
    "LABEL_6": "정치"
}

# 테스트 문장
test_texts = [
    "올해 외국인 주식 투자 성적, 개미 수익률의 4배로 압승.",  # 경제
    "성적은 4위 하지만 인기는 1위 삼성라이온즈, 2025프로야구 관중 순위, 82.5%라는 미친 좌석 점유율.",  # 스포츠
    "대통령 '한글이 그린 세상, 국민이 주인인 나라의 또 다른 모습'.",  # 정치
    "AI 붐타고 '제 2전성기' 맞은 이 회사... 클라우드 업계 엔비디아 될 수 있을까",  # IT과학
    "고속도로 실시간 교통상황 '부산->서울 6시간 40분'... 오후 4~5시 절정.",  # 생활문화
    "10대 사망 최대 원인은 '자살'...10대도 심리부검 검토해야"  # 사회
]

# 예측 실행
for text in test_texts:
    result = classifier(text)[0]
    label_name = label_map[result["label"]]
    print(f"📰 입력 문장: {text}")
    print(f"📌 예측 결과: {label_name} ({result['score']:.2f})")
    print("-" * 60)

## Trainer를 사용하지 않는 학습: (1) 학습을 위한 모델과 토크나이저 준비

In [ ]:
import torch
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from torch.optim import AdamW

def tokenize_function(examples): # 제목(title) 컬럼에 대한 토큰화
    return tokenizer(examples["title"], padding="max_length", truncation=True)

# 모델과 토크나이저 불러오기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_id = "klue/roberta-base"
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=len(train_dataset.features['label'].names))
tokenizer = AutoTokenizer.from_pretrained(model_id)
model.to(device)

## Trainer를 사용하지 않는 학습: (2) 학습을 위한 데이터 준비

In [ ]:
def make_dataloader(dataset, batch_size, shuffle=True):
    dataset = dataset.map(tokenize_function, batched=True).with_format("torch") # 데이터셋에 토큰화 수행
    dataset = dataset.rename_column("label", "labels") # 컬럼 이름 변경
    dataset = dataset.remove_columns(column_names=['title']) # 불필요한 컬럼 제거
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

# 데이터로더 만들기
train_dataloader = make_dataloader(train_dataset, batch_size=8, shuffle=True)
valid_dataloader = make_dataloader(valid_dataset, batch_size=8, shuffle=False)
test_dataloader = make_dataloader(test_dataset, batch_size=8, shuffle=False)

## Trainer를 사용하지 않는 학습: (3) 학습을 위한 함수 정의



In [ ]:
def train_epoch(model, data_loader, optimizer):
    model.train()
    total_loss = 0
    for batch in tqdm(data_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device) # 모델에 입력할 토큰 아이디
        attention_mask = batch['attention_mask'].to(device) # 모델에 입력할 어텐션 마스크
        labels = batch['labels'].to(device) # 모델에 입력할 레이블
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels) # 모델 계산
        loss = outputs.loss # 손실
        loss.backward() # 역전파
        optimizer.step() # 모델 업데이트
        total_loss += loss.item()
    avg_loss = total_loss / len(data_loader)
    return avg_loss

## Trainer를 사용하지 않는 학습: (4) 평가를 위한 함수 정의

In [ ]:
def evaluate(model, data_loader):
    model.eval()
    total_loss = 0
    predictions = []
    true_labels = []
    with torch.no_grad():
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits
            loss = outputs.loss
            total_loss += loss.item()
            preds = torch.argmax(logits, dim=-1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    avg_loss = total_loss / len(data_loader)
    accuracy = np.mean(np.asarray(predictions) == np.asarray(true_labels))
    return avg_loss, accuracy

## Trainer를 사용하지 않는 학습: (5) 학습 수행

In [ ]:
num_epochs = 1
optimizer = AdamW(model.parameters(), lr=5e-5)

# 학습 루프
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    train_loss = train_epoch(model, train_dataloader, optimizer)
    print(f"Training loss: {train_loss}")
    valid_loss, valid_accuracy = evaluate(model, valid_dataloader)
    print(f"Validation loss: {valid_loss}")
    print(f"Validation accuracy: {valid_accuracy}")

# Testing
_, test_accuracy = evaluate(model, test_dataloader)
print(f"Test accuracy: {test_accuracy}") # 정확도 0.82

## 허깅페이스 허브에 모델 업로드

In [ ]:
# 모델의 예측 아이디와 문자열 레이블을 연결할 데이터를 모델 config에 저장
id2label = {i: label for i, label in enumerate(train_dataset.features['label'].names)}
label2id = {label: i for i, label in id2label.items()}
model.config.id2label = id2label
model.config.label2id = label2id

In [ ]:
from huggingface_hub import login

login(token="본인허깅페이스토큰")
repo_id = f"허깅페이스ID/roberta-base-klue-ynat-classification"
# Trainer를 사용한 경우
trainer.push_to_hub(repo_id)
# 직접 학습한 경우
model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)